In [ ]:
!pip install textblob
!pip install -U scikit-learn
from google.colab import drive

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder # Import OneHotEncoder
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.compose import ColumnTransformer # Import ColumnTransformer
from textblob import TextBlob



def mount_drive():
    drive.mount('/content/drive', force_remount=True)

def load_data(file_path, sample_size=1000):
    df = pd.read_csv(file_path, low_memory=False).sample(n=sample_size, random_state=42)
    df.rename(columns={'comments_orig': 'comments'}, inplace=True)

    df['clean_title'] = df['clean_title'].astype(str).str.lower()
    df['comments'] = df['comments'].astype(str)
    df['separated_comment'] = ''  # Initialize as empty strings
    return df

def get_sentiment(text):
    blob = TextBlob(text)
    return blob.sentiment.polarity, blob.sentiment.subjectivity

def process_comments(row):
    try:
        comments = row['comments'].split('|__|')
    except AttributeError:
        return 0, 0
    polarities, subjectivities = zip(*[get_sentiment(str(comment)) for comment in comments]) if comments else ([0], [0])
    return np.mean(polarities), np.mean(subjectivities)

def preprocess_data(df):
    df['comments_polarity'], df['comments_subjectivity'] = zip(*df.apply(process_comments, axis=1))
    df['clean_title_polarity'], df['clean_title_subjectivity'] = zip(*df['clean_title'].apply(get_sentiment))
    df['separated_comment'] = df['comments'].apply(lambda x: x.split('|__|'))
    return df

def prepare_features_and_labels(df):
    # Select features for the model
    X = df[['clean_title', 'score','separated_comment', 'num_comments', 'upvote_ratio']]

    # *** Handle missing values (NaNs) *** Impute numerical features first
    numerical_features = ['score', 'num_comments', 'upvote_ratio']
    X[numerical_features] = X[numerical_features].fillna(X[numerical_features].mean())


    # Target variable
    y = df['2_way_label'].astype(str)
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(y)


    # Split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    # Create a ColumnTransformer to apply TF-IDF to text data,
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', 'passthrough', ['score', 'num_comments', 'upvote_ratio']),
            ('text', TfidfVectorizer(max_features=5000), 'clean_title'),
            #('cat', OneHotEncoder(handle_unknown='ignore'), ['subreddit']), # Applying OneHotEncoder to 'subreddit'
        ])

    # Fit and transform the training data
    X_train = preprocessor.fit_transform(X_train)
    X_test = preprocessor.transform(X_test)

    # *** Clip negative values to 0 ***
    # This ensures all values passed to MultinomialNB are non-negative
    X_train = X_train.toarray()  # Convert to dense array if necessary
    X_test = X_test.toarray()    # Convert to dense array if necessary

    X_train[X_train < 0] = 0
    X_test[X_test < 0] = 0

    return (X_train, X_test, y_train, y_test), label_encoder, preprocessor # Return preprocessor


def build_pipeline():
    # No changes here, the pipeline uses the preprocessor now
    return Pipeline([
        ('classifier', MultinomialNB())
    ])

def evaluate_model(model, X_test, y_test, label_encoder):
    y_pred = model.predict(X_test)
    print("Classification Report:")
    unique_labels = np.unique(y_test)
    target_names = label_encoder.classes_[unique_labels]
    print(classification_report(y_test, y_pred, target_names=target_names))
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))
    print(f"Test Accuracy: {accuracy_score(y_test, y_pred)}")

def predict_new_comments(model, label_encoder, preprocessor, comments, title):
    # Preprocess the new comments using the same preprocessor
    comments_transformed = preprocessor.transform(pd.DataFrame({'clean_title':title * len(comments),
                                                                'score': [0] * len(comments),
                                                                'num_comments': [0] * len(comments),
                                                                'upvote_ratio': [0] * len(comments),
                                                                'separated_comment': [''] * len(comments)}))
    predictions = model.predict(comments_transformed)
    return label_encoder.inverse_transform(predictions)

def main():
    mount_drive()
    file_path = "/content/drive/My Drive/datasets/merged_cleaned_data_v34_no_photo.csv"
    df = preprocess_data(load_data(file_path))
    (X_train, X_test, y_train, y_test), label_encoder, preprocessor = prepare_features_and_labels(df) # Get preprocessor
    model = build_pipeline()
    model.fit(X_train, y_train)
    evaluate_model(model, X_test, y_test, label_encoder)
    new_title = "Shocking Discovery Leaves Internet Divided—Hoax or Breakthrough?"
    new_comments = ["This is outrageous! There’s no way this can be true!", "I can’t believe people are falling for this. It’s just a hoax."]
    # Pass the preprocessor to predict_new_comments
    print("Predictions for new comments:", predict_new_comments(model, label_encoder, preprocessor, new_comments, new_title))

if __name__ == "__main__":
    main()

Mounted at /content/drive
Classification Report:
              precision    recall  f1-score   support

           0       0.50      0.08      0.14        59
           1       0.72      0.96      0.82       141

    accuracy                           0.70       200
   macro avg       0.61      0.52      0.48       200
weighted avg       0.65      0.70      0.62       200

Confusion Matrix:
[[  5  54]
 [  5 136]]
Test Accuracy: 0.705
Predictions for new comments: ['0' '0']


<ipython-input-5-045e33945e8d>:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[numerical_features] = X[numerical_features].fillna(X[numerical_features].mean())
